In [8]:
# kernel = Python 3.11.4
import cv2 as cv
import torch
import tensorflow as tf
import numpy as np

In [7]:
# check version
print(cv.__version__)
print(torch.__version__)
print(tf.__version__)

4.8.0
2.0.1
2.13.0


# Import Videos
```python

# can set the PATH to 0 for using webcam
cap = cv.vediocapture("PATH")
# set the width and height
cap.set(3, 640) # 3 is the id number
cap.set(4, 480)
# set the brightness
cap.set(10, 100)

while True:
    success, img = cap.read()
    cv.imshow("Video", img)
    # if q is pressed, break
    # waitkey(1) is a delay to close the window for 1ms
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

```

# Basic Functions

```python

img = cv.imread("PATH")
kernel = np.ones((5, 5), np.uint8)

# convert to gray scale
imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
cv.imshow("Gray Image", imgGray)

# blur the image
imgBlur = cv.GaussianBlur(imgGray, (7, 7), 0)
cv.imshow("Blur Image", imgBlur)

# edge detector
imgCanny = cv.Canny(img, 150, 200)
cv.imshow("Canny Image", imgCanny)

# dilate the image
# thickness of the edge
kernel = np.ones((5, 5), np.uint8)
imgDialation = cv.dilate(imgCanny, kernel, iterations=1)
cv.imshow("Dialation Image", imgDialation)

# erode the image
imgEroded = cv.erode(imgDialation, kernel, iterations=1)
cv.imshow("Eroded Image", imgEroded)

# resize the image
imgResize = cv.resize(img, (300, 200))
cv.imshow("Resized Image", imgResize)

# crop the image
imgCropped = img[0:200, 200:500]
cv.imshow("Cropped Image", imgCropped)

cv.waitKey(0)
```

# Resizing and Cropping

```python
img = cv.imread("PATH")
# know the current size
print(img.shape)
# resize the image
imgResize = cv.resize(img, (300, 200)) # (width, height)
cv.imshow("Resized Image", imgResize)

# crop the image
imgCropped = img[0:200, 200:500] # (height, width)
cv.imshow("Cropped Image", imgCropped)

cv.waitKey(0)
```



# Shape detecting

```python
# create a matrix full of zeros
# current image is black
img = np.zeros((512, 512, 3), np.uint8)
# if we want a blue image
img[:] = 255, 0, 0 # full image in blue
img[200:300, 100:300] = 255, 0, 0 # part of the image in blue
# draw a line
cv.line(img, (0, 0), (300, 300), (0, 255, 0), 3) # (image, start, end, color, thickness) image.shape[1] is the width image.shape[0] is the height
# draw a rectangle
cv.rectangle(img, (0, 0), (250, 350), (0, 0, 255), 2) # (image, start, end, color, thickness) we can change 2 into cv.FILLED to fill the rectangle
# draw a circle
cv.circle(img, (400, 50), 30, (255, 255, 0), 5) # (image, center, radius, color, thickness)
# put text
cv.putText(img, " OPENCV ", (300, 200), cv.FONT_HERSHEY_COMPLEX, 1, (0, 150, 0), 1) # (image, text, start, font, scale, color, thickness)



In [6]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import argparse
import imutils
import cv2
import numpy as np

### Color Detection

In [26]:

def empty(a):
    pass

def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver



# path = '/Users/huxinyu/Desktop/UROP2023/blue21.jpg'
# path = '/Users/huxinyu/Desktop/UROP2023/lambo.png'
# path = '/Users/huxinyu/Desktop/UROP2023/orange.jpg'
# path = '/Users/huxinyu/Desktop/UROP2023/allcolor.jpg'


# create a window named "TrackBars" for adjusting the HSV values
cv2.namedWindow("TrackBars")
cv2.resizeWindow("TrackBars",640,240)
cv2.createTrackbar("Hue Min","TrackBars",0,179,empty)
cv2.createTrackbar("Hue Max","TrackBars",179,179,empty)
cv2.createTrackbar("Sat Min","TrackBars",112,255,empty)
cv2.createTrackbar("Sat Max","TrackBars",255,255,empty)
cv2.createTrackbar("Val Min","TrackBars",0,255,empty)
cv2.createTrackbar("Val Max","TrackBars",255,255,empty)

# A loop to keep the window open and update the HSV values
# until we press the "q" key on our keyboard
while True:
    img = cv2.imread(path)
    imgHSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    h_min = cv2.getTrackbarPos("Hue Min","TrackBars")
    h_max = cv2.getTrackbarPos("Hue Max", "TrackBars")
    s_min = cv2.getTrackbarPos("Sat Min", "TrackBars")
    s_max = cv2.getTrackbarPos("Sat Max", "TrackBars")
    v_min = cv2.getTrackbarPos("Val Min", "TrackBars")
    v_max = cv2.getTrackbarPos("Val Max", "TrackBars")
    # find the best HSV values
    print(h_min,h_max,s_min,s_max,v_min,v_max)
    lower = np.array([h_min,s_min,v_min])
    upper = np.array([h_max,s_max,v_max])
    mask = cv2.inRange(imgHSV,lower,upper)
    imgResult = cv2.bitwise_and(img,img,mask=mask)
    

    # cv2.imshow("Original",img)
    # cv2.imshow("HSV",imgHSV)
    # cv2.imshow("Mask", mask)
    # cv2.imshow("Result", imgResult)

    imgStack = stackImages(0.6,([img,imgHSV],[mask,imgResult]))
    cv2.imshow("Stacked Images", imgStack)

    cv2.waitKey(1)


0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255
0 179 112 255 0 255


KeyboardInterrupt: 

# Shape detecting
